In [1]:
from senticnet.senticnet import SenticNet
import nltk
from tqdm import tqdm
import xlrd
import pandas as pd
import random
from textblob import TextBlob
import numpy as np
import IPython
import re
from nltk.corpus import stopwords
import json
import inflection as inf
from wordcloud import WordCloud
from gensim.models import Word2Vec
import enchant
import time
from nltk import sent_tokenize

# import matplotlib.pyplot as plt

adj = ['JJ','JJR','JJS']
adv = ['RB','RBR','RBS']
vb = ['VB','VBD','VBG','VBN','VBP','VBZ']
nn = ['NN','NNS']


def review_to_words(review_text): 
    if '(Reuters) -' in review_text:
        review_text = review_text.split('(Reuters) -')[1]
    if '*' in review_text:
        review_text = review_text.split('*')[1]
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.split()                             
    _stopwords = set(stopwords.words("english"))
    _stopwords = nltk.corpus.stopwords.words('english')
    _stopwords.append('would')
    _stopwords.append('kmh')
    _stopwords.append('mph')
    _stopwords.append('  ')
    _stopwords.append('Reuters')
    _stopwords.append('reuters')
    # _stopwords = []
    meaningful_words = [w for w in words if w not in _stopwords]
    return meaningful_words

# d = enchant.Dict("en_US")
import spacy
nlp = spacy.load('en')
def stem_and_check(word):
    word = nlp(word)
    return word[0].lemma_
    # word = inf.singularize(word)
    # word = nltk.PorterStemmer().stem(word)
    # if d.check(word):
    #    return word
    # suggest_words = d.suggest(word)
    # if len(suggest_words) == 0:
    #    return word
    # return suggest_words[0]

def my_read(path):
    file = open(path)
    words = []
    for line in file.readlines():
        words.append(line.strip())
    return words

def output_cloud(count,name):
    # 云图
    text = '' 
    for key,value in count.items():
        text += (key+' ') * (value)
    wc = WordCloud(
        width=1000,
        height=600,
        max_font_size=100,      #字体大小
        min_font_size=10,
        collocations=False, 
        max_words=1000
    )
    wc.generate(text)
    wc.to_file(name+'.png') #图片保存

sn = SenticNet()
# concept_info = sn.concept('love')
# polarity_value = sn.polarity_value('love')
# polarity_intense = sn.polarity_intense('love')
# moodtags = sn.moodtags('love')
# semantics = sn.semantics('love')
# sentics = sn.sentics('love') 
 
def sentiment_score(text):
    t = TextBlob(text)
    score = t.sentiment.polarity
    return score
    # tokens = nltk.word_tokenize(text)
    # pos_tags = nltk.pos_tag(tokens)
    # score = 0
    # count = 0
    # for word,tag in pos_tags:
    #   if word in sn.data.keys():
    #    score += float(sn.polarity_intense(word))
    #    count += 1
    #    # print(word,sn.polarity_intense(word))
    # if count == 0: #mid
    #   return -1 
    # return score/count

# test sentiment_score accuracy
# scores = []
# workbook = pd.read_csv(u'sentiment.csv',encoding='ISO-8859-1')
# correct = 0
# pos_count = 0
# neg_count = 0
# pos_correct = 0
# neg_correct = 0

# for i in tqdm(range(0,10000)):
#   i = int(random.random()*1599999)
#   if workbook.loc[i][0] == 0:
#    neg_count += 1
#    if workbook.loc[i][0] == sentiment_score_list(workbook.loc[i][5]):
#     neg_correct += 1
#   elif workbook.loc[i][0] == 4:
#    pos_count += 1
#    if workbook.loc[i][0] == sentiment_score_list(workbook.loc[i][5]):
#     pos_correct += 1
# print('pos',pos_correct/pos_count,pos_count,pos_correct)
# print('neg',neg_correct/neg_count,neg_count,neg_correct)


In [2]:
    path2018 = r'/Users/wangfeihong/Desktop/Dynamic-Financial-News-Collection-and-Analysis/data/labeled_data2018.xls'
    path2019 = r'/Users/wangfeihong/Desktop/Dynamic-Financial-News-Collection-and-Analysis/data/labeled_data2019.xls'

    workbook = xlrd.open_workbook(path2018)
    worksheet = workbook.sheet_by_index(0)
    contents = worksheet.col_values(1)
    companies = worksheet.col_values(2)
    prices = worksheet.col_values(3)
    dates = worksheet.col_values(4)
    rates = []
    score_list = []

In [3]:
datas = []
for i in tqdm(range(0,len(contents))):
    # if '*' not in contents[i]:
        # if companies[i] != 'Apple Inc.':
        #     continue
    price_list = json.loads(prices[i])   
    if price_list[2] == price_list[1]:
        continue
    data = {}
    data['content'] = contents[i]
    sents = sent_tokenize(data['content'])
    data['tokens'] = []
    data['tags'] = []
    for sent in sents:
        token = review_to_words(sent) # 去停用词会影响词性标注吗？？
        data['tokens'].append(token)
        data['tags'].append(nltk.pos_tag(token))
    data['company'] = companies[i]
    data['rate'] = (price_list[2]-price_list[1])/price_list[1]
    data['date'] = dates[i]
    datas.append(data)

    
    


100%|██████████| 20961/20961 [10:41<00:00, 32.66it/s]


In [32]:
workbook = xlrd.open_workbook(path2019)
worksheet = workbook.sheet_by_index(0)
contents = worksheet.col_values(1)
companies = worksheet.col_values(2)
prices = worksheet.col_values(3)
dates = worksheet.col_values(4)
rates = []
score_list = []
datas2 = []
for i in tqdm(range(0,len(contents))):
    # if '*' not in contents[i]:
        # if companies[i] != 'Apple Inc.':
        #     continue
    price_list = json.loads(prices[i]) 
    if price_list[2] == price_list[1]:
        continue
    data = {}
    data['content'] = contents[i]
    sents = sent_tokenize(data['content'])
    data['tokens'] = []
    data['tags'] = []
    for sent in sents:
        token = review_to_words(sent) # 去停用词会影响词性标注吗？？
        data['tokens'].append(token)
        data['tags'].append(nltk.pos_tag(token))
    data['company'] = companies[i]       
    data['rate'] = (price_list[2]-price_list[1])/price_list[1]
    data['date'] = dates[i]
    datas2.append(data)

## sort by time
# dates = {}
# for i in tqdm(range(0,len(_datas))):
#     dates[i] = int(time.mktime(time.strptime(_datas[i]['date'], "%Y-%m-%d")))
# res = sorted(dates.items(),key=lambda dates:dates[1],reverse=False)
# datas = []
# for idx,date in res:
#     datas.append(_datas[idx])

count = {}

POS = 0
NEG = 0

pos_count = 0
neg_count = 0

N = 0 #len of tokens


100%|██████████| 4803/4803 [02:26<00:00, 32.87it/s]


In [33]:
for data in tqdm(datas):
    for tokens in data['tokens']: 
        N += len(tokens)
        rate = data['rate'] # 选当天的股票变化判断涨跌，因为相关度当天的最高
        if rate>0:
            pos_count += 1
            POS += len(tokens)
            for token in tokens:
                if len(token) < 3:
                    continue
                if 'not' in tokens:
                    token = 'not_'+token
                if token in count.keys():
                    count[token]['pos'] += 1
                    count[token]['pos_rate'] += rate
                else:
                    count[token] = {'pos':1,'neg':0,'pos_rate':rate,'neg_rate':0} 

        if rate<0:
            neg_count += 1
            NEG += len(tokens)
            for token in tokens:
                if len(token) < 3:
                    continue
                if 'not' in tokens:
                    token = 'not_'+token
                if token in count.keys():
                    count[token]['neg'] += 1
                    count[token]['neg_rate'] -= rate
                else:
                    count[token] = {'pos':0,'neg':1,'pos_rate':0,'neg_rate':-rate}



100%|██████████| 20819/20819 [00:07<00:00, 2831.45it/s]


In [35]:
## freq
copy = count.copy()
sent_words = [] # PD>0.3情感值

freq_pos = pos_count/len(datas) 
freq_neg = neg_count/len(datas)

# DS sent and PMI sent
for word,value in tqdm(copy.items()):
    if value['pos']+value['neg']<20:
        del count[word]
        continue
    freq_w_pos = value['pos']/len(datas)
    freq_w_neg = value['neg']/len(datas)
    freq_w = (value['pos']+value['neg'])/len(datas)
    if freq_w_pos*N == 0:
        PMI_w_pos = 0
    else:
        PMI_w_pos = np.log2(freq_w_pos*N/freq_w*freq_pos)
    if freq_w_neg*N == 0:
        PMI_w_neg = 0
    else:
        PMI_w_neg = np.log2(freq_w_neg*N/freq_w*freq_neg)
    count[word]['PMI_sent'] = PMI_w_pos - PMI_w_neg

    pos = value['pos']/len(datas)
    neg = value['neg']/len(datas)
    value['PD'] = (pos-neg)/(pos+neg) # polarity difference
    if abs(value['PD']) > 0.3 and nltk.pos_tag([word])[0][1] in adj+adv:  
        sent_words.append(word)
    count[word]['sent'] = value['PD']*value['PD'] * np.sign(value['PD'])

    pos_rate = value['pos_rate']/len(datas)
    neg_rate = value['neg_rate']/len(datas)
    value['PD_rate'] = (pos_rate-neg_rate)/(pos_rate+neg_rate) # polarity difference
    count[word]['sent_rate'] = value['PD_rate']*value['PD_rate'] * np.sign(value['PD_rate'])


100%|██████████| 59244/59244 [00:00<00:00, 80496.98it/s]


In [36]:
res = sorted(count.items(),key=lambda count:count[1]['sent'],reverse=False)
for r in res[:100]:
    print(r[0],r[1]['sent'])
res = sorted(count.items(),key=lambda count:count[1]['sent'],reverse=True)  
for r in res[:100]:
    print(r[0],r[1]['sent'])

IOC -1.0
Liveris -1.0
Signa -1.0
Kaufhof -1.0
Eurosport -1.0
Ciarallo -1.0
Morneau -1.0
Contrafund -1.0
Solodyn -1.0
TCH -1.0
Tenable -1.0
CITs -1.0
Players -1.0
Havilland -1.0
ITEP -1.0
Genuine -1.0
Subsea -1.0
GAZ -1.0
SoCalGas -1.0
GreenSky -1.0
reconstitution -1.0
Ontex -1.0
Pick -1.0
Energen -1.0
PBGC -1.0
Cristi -1.0
Multichoice -1.0
Marketo -1.0
Brii -1.0
Yao -1.0
Binladin -1.0
implanted -1.0
Hetrick -1.0
Wasp -1.0
Perryman -1.0
Stimwave -1.0
neuromodulation -1.0
Skinsei -1.0
Ghosn -0.87890625
Tianjin -0.8751300728407907
Shave -0.8622448979591836
Cineworld -0.8573388203017833
Springer -0.8573388203017833
Halo -0.8463999999999998
McKenzie -0.8402777777777777
Encana -0.8336483931947068
Celltrion -0.8336483931947068
rats -0.8264462809917354
Temenos -0.8185941043083901
PJM -0.8185941043083901
Essendant -0.8185941043083901
Goh -0.8099999999999998
Newfield -0.795471146822498
Catalyst -0.7809626825310977
bows -0.7656249999999998
tournament -0.7585848074921955
Marubeni -0.75858480749219

In [37]:
# res = sorted(count.items(),key=lambda count:count[1]['PD'],reverse=True)
# print(res)

## neg pos 词
# pos_words = {}
# neg_words = {}
# for word in sent_words:
#    if count[word]['sent'] > 0:
#       pos_words[word.lower()] = count[word]['pos']+count[word]['neg']
#    else:
#       neg_words[word.lower()] = count[word]['pos']+count[word]['neg']

# output_cloud(pos_words,'pos')
# output_cloud(neg_words,'neg')

In [38]:
## 求于bl词典的覆盖率
bl_sent = {}
bl_pos = my_read(r'/Users/wangfeihong/Desktop/Dynamic-Financial-News-Collection-and-Analysis/sentiment_analysis/bl/positive.txt')  # 4783
bl_neg = my_read(r'/Users/wangfeihong/Desktop/Dynamic-Financial-News-Collection-and-Analysis/sentiment_analysis/bl/negative.txt')  # 2006


for word in bl_pos:
    bl_sent[word] = 1
for word in bl_pos:
    bl_sent[word] = -1

In [39]:
# pc = 0
# for word in pos_words:
#    if word in bl_pos:
#       pc += 1
# pos_accuracy = pc/len(pos_words)  # 0.2857142857142857

# nc = 0
# for word in neg_words:
#    if word in bl_neg:
#       nc += 1
# neg_accuracy = nc/len(neg_words)  # 0.18


In [40]:
## context sentiment dict
sent_words = [word.lower() for word in sent_words]
feature_words = {}
sentiment_feature = {}

for data in tqdm(datas):
    for tags in data['tags']:
        for word,tag in tags:
            if tag not in nn or len(word)<3: # vb+nn
                continue
            # word = stem_and_check(word)
            if word not in feature_words.keys():
                feature_words[word] = 1
            else:
                feature_words[word] += 1

# avg_f = sum([item[1] for item in feature_words.items()])/len(feature_words.keys())
res = sorted(feature_words.items(),key=lambda feature_words:feature_words[1],reverse=True)
words = res[:400]
# for word,value in tqdm(copy.items()):
#     if value<avg_f+200:
#         del feature_words[word]

feature_words = [inf.singularize(word).lower() for word,freq in words]

100%|██████████| 20819/20819 [00:01<00:00, 11628.05it/s]


In [41]:
# sf_len = 0
# for data in tqdm(datas):
#     rate = data['rate']
#     for tokens in data['tokens']:
#         token_dict = {}
#         for token in tokens:
#             token_dict[inf.singularize(token).lower()] = token
#         _tokens = [inf.singularize(token).lower() for token in tokens]

#         for w in list(set(sent_words).intersection(set(_tokens))):
#             for f in list(set(feature_words).intersection(set(_tokens))):
#                 if f != w:
#                     if abs(_tokens.index(w)-_tokens.index(f))<3 and ',' not in data['content'][min(data['content'].index(token_dict[f]),data['content'].index(token_dict[w])):max(data['content'].index(token_dict[f]),data['content'].index(token_dict[w]))]:
#                         sf_len += 1
#                         if f not in sentiment_feature.keys():
#                             sentiment_feature[f] = {}
#                             if rate > 0:
#                                 sentiment_feature[f][w] = {'pos':1,'neg':0}
#                             if rate < 0:
#                                 sentiment_feature[f][w] = {'pos':0,'neg':1}
#                         else:
#                             if w not in sentiment_feature[f].keys():
#                                 sentiment_feature[f][w] = {'pos':0,'neg':0}
#                             if rate > 0:
#                                 sentiment_feature[f][w]['pos'] += 1
#                             if rate < 0:
#                                 sentiment_feature[f][w]['neg'] += 1

# avg_sf = sf_len/len(sentiment_feature.keys())
# copy = sentiment_feature.copy()

In [42]:
# Predict
# content to vector

for data in tqdm(datas):
    idx = datas.index(data)
    tokens = data['tokens']
    datas[idx]['DsVector'] = [0,0,0,0]
    datas[idx]['DsVector_rate'] = [0,0,0,0]
    datas[idx]['SnVector'] = [0,0,0,0]
    datas[idx]['BlVector'] = [0,0,0,0]
    datas[idx]['PmiVector'] = [0,0,0,0]
    datas[idx]['ContextVector'] = [0,0,0,0]
    
    # for f in [token for token in tokens if token in sentiment_feature.keys()]:
    #     for w in sentiment_feature[f].keys():
    #         if w in tokens:
    #             if abs(tokens.index(f)-tokens.index(w))<3 and ',' not in data['content'][min(data['content'].index(f),data['content'].index(w)):max(data['content'].index(f),data['content'].index(w))]:
    #                 if tags[tokens.index(f)][1] in adj:
    #                     if f in count.keys():
    #                         datas[idx]['ContextVector'][0] += count[f]['sent']
    #                 elif tags[tokens.index(f)][1] in adv:
    #                     if f in count.keys():
    #                         datas[idx]['ContextVector'][1] += count[f]['sent']
    #                 if tags[tokens.index(w)][1] in nn:
    #                     if w in count.keys():
    #                         datas[idx]['ContextVector'][2] = count[w]['sent']
    #                 elif tags[tokens.index(w)][1] in vb:
    #                     if w in count.keys():
    #                         datas[idx]['ContextVector'][3] = count[w]['sent']
    #                 # print(sentiment_feature[f][w]['sent'])
    for tags in data['tags']:
        for word,tag in tags:
            if tag in adj:
                if word in count.keys():
                    datas[idx]['DsVector'][0] += count[word]['sent']
                    datas[idx]['DsVector_rate'][0] += count[word]['sent_rate']
                    datas[idx]['PmiVector'][0] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas[idx]['SnVector'][0] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas[idx]['BlVector'][0] += bl_sent[word]
            elif tag in adv:
                if word in count.keys():
                    datas[idx]['SnVector'][1] += count[word]['sent']
                    datas[idx]['DsVector_rate'][1] += count[word]['sent_rate']
                    datas[idx]['PmiVector'][1] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas[idx]['SnVector'][1] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas[idx]['BlVector'][1] += bl_sent[word]  
            elif tag in nn:
                if word in count.keys():
                    datas[idx]['DsVector'][2] = count[word]['sent']
                    datas[idx]['DsVector_rate'][2] += count[word]['sent_rate']
                    datas[idx]['PmiVector'][2] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas[idx]['SnVector'][2] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas[idx]['BlVector'][2] += bl_sent[word]
            elif tag in vb:
                if word in count.keys():
                    datas[idx]['DsVector'][3] = count[word]['sent']
                    datas[idx]['DsVector_rate'][3] += count[word]['sent_rate']
                    datas[idx]['PmiVector'][3] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas[idx]['SnVector'][3] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas[idx]['BlVector'][3] += bl_sent[word]
    # datas[idx]['DsVector'] = [adv_score,adv_score,noun_score,verb_score]


100%|██████████| 20819/20819 [00:34<00:00, 595.72it/s]


In [43]:
# Predict
# content to vector

for data in tqdm(datas2):
    idx = datas2.index(data)
    tokens = data['tokens']
    datas2[idx]['DsVector'] = [0,0,0,0]
    datas2[idx]['DsVector_rate'] = [0,0,0,0]
    datas2[idx]['SnVector'] = [0,0,0,0]
    datas2[idx]['BlVector'] = [0,0,0,0]
    datas2[idx]['PmiVector'] = [0,0,0,0]
    datas2[idx]['ContextVector'] = [0,0,0,0]
    
    # for f in [token for token in tokens if token in sentiment_feature.keys()]:
    #     for w in sentiment_feature[f].keys():
    #         if w in tokens:
    #             if abs(tokens.index(f)-tokens.index(w))<3 and ',' not in data['content'][min(data['content'].index(f),data['content'].index(w)):max(data['content'].index(f),data['content'].index(w))]:
    #                 if tags[tokens.index(f)][1] in adj:
    #                     if f in count.keys():
    #                         datas[idx]['ContextVector'][0] += count[f]['sent']
    #                 elif tags[tokens.index(f)][1] in adv:
    #                     if f in count.keys():
    #                         datas[idx]['ContextVector'][1] += count[f]['sent']
    #                 if tags[tokens.index(w)][1] in nn:
    #                     if w in count.keys():
    #                         datas[idx]['ContextVector'][2] = count[w]['sent']
    #                 elif tags[tokens.index(w)][1] in vb:
    #                     if w in count.keys():
    #                         datas[idx]['ContextVector'][3] = count[w]['sent']
    #                 # print(sentiment_feature[f][w]['sent'])
    for tags in data['tags']:
        for word,tag in tags:
            if tag in adj:
                if word in count.keys():
                    datas2[idx]['DsVector'][0] += count[word]['sent']
                    datas2[idx]['DsVector_rate'][0] += count[word]['sent_rate']
                    datas2[idx]['PmiVector'][0] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas2[idx]['SnVector'][0] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas2[idx]['BlVector'][0] += bl_sent[word]
            elif tag in adv:
                if word in count.keys():
                    datas2[idx]['SnVector'][1] += count[word]['sent']
                    datas2[idx]['DsVector_rate'][1] += count[word]['sent_rate']
                    datas2[idx]['PmiVector'][1] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas2[idx]['SnVector'][1] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas2[idx]['BlVector'][1] += bl_sent[word]  
            elif tag in nn:
                if word in count.keys():
                    datas2[idx]['DsVector'][2] = count[word]['sent']
                    datas2[idx]['DsVector_rate'][2] += count[word]['sent_rate']
                    datas2[idx]['PmiVector'][2] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas2[idx]['SnVector'][2] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas2[idx]['BlVector'][2] += bl_sent[word]
            elif tag in vb:
                if word in count.keys():
                    datas2[idx]['DsVector'][3] = count[word]['sent']
                    datas2[idx]['DsVector_rate'][3] += count[word]['sent_rate']
                    datas2[idx]['PmiVector'][3] += count[word]['PMI_sent']
                if word in sn.data.keys():
                    datas2[idx]['SnVector'][3] += float(sn.polarity_intense(word))
                if word in bl_sent.keys():
                    datas2[idx]['BlVector'][3] += bl_sent[word]
    # datas[idx]['DsVector'] = [adv_score,adv_score,noun_score,verb_score]


100%|██████████| 4776/4776 [00:04<00:00, 1132.37it/s]


In [44]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from keras.utils import to_categorical
import matplotlib.pyplot as plt

X = [data['DsVector'] for data in datas]
# X = [data['SnVector'] for data in datas]
# X = [data['BlVector'] for data in datas]
# X = [data['PmiVector'] for data in datas]
# X = [data['ContextVector'] for data in datas]
X = [data['DsVector_rate'] for data in datas]
Y = [np.sign(data['rate']) for data in datas]

# Y = [data['rate'] for data in datas]

train_x,test_x,train_y,test_y = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
# # x_train = X[:2500]
# # y_train = Y[:2500]
# # x_test = X[-300:]
# # y_test = Y[-300:]
clf = GaussianNB()
# clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('准确率：',clf.score(np.array(test_x), np.array(test_y))) 
print('召回率：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('精确率：',precision_score(test_y, clf.predict(test_x), average='macro'))

准确率： 0.6621037463976945
召回率： 0.6664047951582868
精确率： 0.6783703703703703


In [45]:
X = [data['DsVector'] for data in datas]
train_x,test_x,train_y,test_y = model_selection.train_test_split(X,Y,test_size=0.2,shuffle=False)
clf = GaussianNB()
# clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('准确率：',clf.score(np.array(test_x), np.array(test_y))) 
print('召回率：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('精确率：',precision_score(test_y, clf.predict(test_x), average='macro'))

准确率： 0.6083093179634966
召回率： 0.6018721734503858
精确率： 0.6226475550920627


In [46]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('准确率：',clf.score(np.array(test_x), np.array(test_y))) 
print('召回率：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('精确率：',precision_score(test_y, clf.predict(test_x), average='macro'))

准确率： 0.654418828049952
召回率： 0.6548713642812805
精确率： 0.6547700396219609


In [47]:
print(datas[0]['date'],datas[-1]['date'],datas2[0]['date'],datas2[-1]['date'])

2018-01-03 2019-01-22 2018-12-31 2019-04-11


In [49]:
# 用18年数据训练，测试19年数据
train_x = [data['DsVector_rate'] for data in datas]
train_y = [np.sign(data['rate']) for data in datas]
test_x = [data['DsVector_rate'] for data in datas2[2000:]]
test_y = [np.sign(data['rate']) for data in datas2[2000:]]
# clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf = GaussianNB()
# clf = LinearRegression()
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('准确率：',clf.score(np.array(test_x), np.array(test_y))) 
print('召回率：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('精确率：',precision_score(test_y, clf.predict(test_x), average='macro'))


准确率： 0.5259365994236311
召回率： 0.4958963157435996
精确率： 0.49491717717352257


In [56]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf.fit(np.array(train_x), np.array(train_y))
predict_y = clf.predict(test_x)
print('准确率：',clf.score(np.array(test_x), np.array(test_y))) 
print('召回率：',recall_score(test_y,clf.predict(test_x),average = 'macro'))
print('精确率：',precision_score(test_y, clf.predict(test_x), average='macro'))

准确率： 0.4989193083573487


ValueError: Classification metrics can't handle a mix of multiclass-multioutput and multilabel-indicator targets

In [53]:
# dnn
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical


train_x = [data['DsVector_rate'] for data in datas]
Y = [np.sign(data['rate']) for data in datas]
test_x = [data['DsVector_rate'] for data in datas2[2000:]]
Y2 = [np.sign(data['rate']) for data in datas2[2000:]]

train_y = []
for y in Y:
    if y == 1:
        train_y.append(np.array([0,1]))
    else:    
        train_y.append(np.array([1,0]))

test_y = []
for y in Y2:
    if y == 1:
        test_y.append(np.array([0,1]))
    else:    
        test_y.append(np.array([1,0]))

num_classes = 2
# test_y = to_categorical(Y,num_classes=3)

nmodel = Sequential()
nmodel.add(Dense(units=num_classes, activation = 'relu', input_dim = np.array(train_x).shape[1]))
nmodel.add(Dropout(0.5))
nmodel.add(Dense(2, activation = 'relu'))
nmodel.add(Dropout(0.5))
# dropout:https://blog.csdn.net/program_developer/article/details/80737724
nmodel.add(Dense(2, activation = 'softmax'))
nmodel.compile(loss = 'categorical_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])
nmodel.fit(np.array(train_x),np.array(train_y),epochs=10, batch_size=5)
nmodel.evaluate(np.array(test_x),np.array(test_y), batch_size=5)

Epoch 1/10
20819/20819 [==============================] - 2s 90us/step - loss: 0.6878 - acc: 0.5282: 0s - loss: 0.6882 - ac
Epoch 2/10
20819/20819 [==============================] - 2s 89us/step - loss: 0.6865 - acc: 0.5299
Epoch 3/10
20819/20819 [==============================] - 2s 89us/step - loss: 0.6863 - acc: 0.5324: 0s - loss: 0.6863 
Epoch 4/10
20819/20819 [==============================] - 2s 88us/step - loss: 0.6843 - acc: 0.5361: 0s - loss: 0
Epoch 5/10
20819/20819 [==============================] - 2s 93us/step - loss: 0.6849 - acc: 0.5366
Epoch 6/10
20819/20819 [==============================] - 2s 88us/step - loss: 0.6855 - acc: 0.5327
Epoch 7/10
20819/20819 [==============================] - 2s 97us/step - loss: 0.6843 - acc: 0.5352
Epoch 8/10
20819/20819 [==============================] - 2s 89us/step - loss: 0.6834 - acc: 0.5395
Epoch 9/10
20819/20819 [==============================] - 2s 88us/step - loss: 0.6845 - acc: 0.5365
Epoch 10/10
2776/2776 [===================

[0.6949739774168396, 0.5108069263946169]

In [51]:
datas2[2000]['date']

'2019-01-29'

In [93]:
import pickle
# output = open('D:\\xxx\\data.pkl', 'wb')
# input = open('D:\\xxx\\data.pkl', 'rb')
# s = pickle.dump(clf, output)
# output.close()
# clf2 = pickle.load(input)
# input.close()
# print clf2.predict(X[0:1])
datas[1]

{'content': 'By Sijia Jiang\nHONG KONG (Reuters) - Huawei Technologies Co Ltd\'s [HWT.UL] planned deal with U.S. carrier AT&T Inc (T.N) to sell its smartphones in the United States has collapsed at the 11th hour because of security concerns, people with knowledge of the matter said, in a blow to the Chinese firm\'s global ambitions.\nAT&T was pressured to drop the deal after members of the U.S. Senate and House intelligence committees sent a letter on Dec. 20 to the U.S. Federal Communications Commission (FCC) citing concerns about Huawei\'s plans to launch consumer products through a major U.S. telecom carrier.\nThe letter to FCC Chairman Ajit Pai, which was signed by 18 lawmakers, noted concerns about Chinese companies in the U.S. telecommunications industry.\nThe letter notes the committee\'s concerns "about Chinese espionage in general, and Huawei\'s role in that espionage in particular." A copy of the letter was seen by Reuters.\nHuawei said in a statement on Tuesday that its flag